## ECG heartbeat classification

### Loading data and libraries

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objects as go
import torch
import matplotlib.pyplot as plt
plotly.offline.init_notebook_mode(connected=True)
import numpy as np

In [ ]:
df_train = pd.read_csv("../ecg_data/ecg/mitbih_train.csv", header=None)
df_test = pd.read_csv("../ecg_data/ecg/mitbih_test.csv", header=None)

In [ ]:
df_train[187] = df_train[187].astype(int)

df_train.head()
df_train.info()
df_train.describe()

In [ ]:
repartition = df_train[187].astype(int).value_counts()
print(repartition)
classes={0:"Normal",
         1:"Artial Premature",
         2:"Premature ventricular contraction",
         3:"Fusion of ventricular and normal",
         4:"Undetermined"}

### Data visualisation

In [ ]:
labels = [classes[i] for i in list(repartition.index)]
values = repartition.values
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_layout(title="Data Partitioning By ECG Signal Type")
fig.update_layout(legend_title='<b> Legend </b>')
fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=18,
    )
)
fig.show()
plotly.offline.plot(fig, filename='graphs/pie.html')

In [ ]:
index_0 = np.argwhere(df_train[187].astype(int).values == 0)[0][0]
index_1 = np.argwhere(df_train[187].astype(int).values == 1)[0][0]
index_2 = np.argwhere(df_train[187].astype(int).values == 2)[0][0]
index_3 = np.argwhere(df_train[187].astype(int).values == 3)[0][0]
index_4 = np.argwhere(df_train[187].astype(int).values == 4)[0][0]
signal_0 = df_train.iloc[index_0].values[:187]
signal_1 = df_train.iloc[index_1].values[:187]
signal_2 = df_train.iloc[index_2].values[:187]
signal_3 = df_train.iloc[index_3].values[:187]
signal_4 = df_train.iloc[index_4].values[:187]

In [ ]:

# sampling frequency 125Hz -> 0.008 inverse sampling frequency
isf = 0.008
x_axis = np.linspace(0, 187, 187) * isf

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=x_axis, y=signal_0, mode='lines+markers', name=classes[0]))
fig.add_trace(go.Scatter(x=x_axis, y=signal_1, mode='lines+markers', name=classes[1]))
fig.add_trace(go.Scatter(x=x_axis, y=signal_2, mode='lines+markers', name=classes[2]))
fig.add_trace(go.Scatter(x=x_axis, y=signal_3, mode='lines+markers', name=classes[3]))
fig.add_trace(go.Scatter(x=x_axis, y=signal_4, mode='lines+markers', name=classes[4]))

fig.update_layout(title='Example ECG Signal Categories', xaxis_title='Time (s)', yaxis_title='Amplitude')
fig.update_layout(legend_title='<b> Legend </b>')
fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=18,
    )
)

fig.show()
plotly.offline.plot(fig, filename='graphs/signals.html')

In [ ]:
torch_tensor = torch.tensor(df_train[df_train.columns[:-1]].values)
torch_labels = torch.tensor(df_train[df_train.columns[-1]].values)
train_dataset = [[torch_tensor[i].float().unsqueeze(0), torch_labels[i].long()] for i in range(torch_tensor.shape[0])]

torch_tensor = torch.tensor(df_test[df_test.columns[:-1]].values)
torch_labels = torch.tensor(df_test[df_test.columns[-1]].values)
test_dataset = [[torch_tensor[i].float().unsqueeze(0), torch_labels[i].long()] for i in range(torch_tensor.shape[0])]

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    def __init__(self):
        super(classifier, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=8, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3))
        self.conv2 = nn.Sequential(
            nn.Conv1d(8, 32, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3))   
        self.fc = nn.Linear(in_features=576, out_features=5)

    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(1, -1)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

In [ ]:
def test(model,data_loader):
    model.train(False)

    running_corrects = 0.0
    running_loss = 0.0
    size = 0

    for data in data_loader:
        inputs, labels = data   
        bs = labels.size(0)
                
        outputs = model(inputs)
        classes = labels
        loss = loss_fn(outputs,classes.type(torch.LongTensor)) 
        _,preds = torch.max(outputs,1)
        running_corrects += torch.sum(preds == classes.data.type(torch.LongTensor))
        running_loss += loss.data
        size += bs
    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))

In [ ]:
def train(model,data_loader,loss_fn,optimizer,n_epochs=1):
    model.train(True)
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            bs = labels.size(0)
            outputs = model(inputs)
            loss = loss_fn(outputs,labels)       
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            _,preds = torch.max(outputs,1)
            running_corrects += torch.sum(preds == labels.data.type(torch.LongTensor))
            running_loss += loss.data
            size += bs
        epoch_loss = running_loss.item() / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        test(model, test_loader)
    return loss_train, acc_train

In [ ]:
conv_class = classifier()
loss_fn = nn.NLLLoss()
learning_rate = 1e-3
optimizer_cl = torch.optim.Adam(conv_class.parameters(), lr=learning_rate)
l_t, a_t = train(conv_class,train_loader,loss_fn,optimizer_cl,n_epochs = 2)